In [1]:
import pandas as pd
import os, pickle
import string
import re
import numpy as np
from sklearn import linear_model
import sklearn.metrics as mcs
from collections import defaultdict
import time
from tqdm import tqdm as timer

### Director rating dictionary

In [3]:
fname = "movie_rating.txt"

movie_rating_dict = {}
line_count = 0

with open(fname, encoding="ISO-8859-1") as fin:
    for line in fin:
        line_count += 1
        obj_list = re.split(r"\t+", line) ############# change to \t        

        obj = obj_list[0]

        p1 = re.search(r"\((\d{4}|\?{4})[^()]*\)(.+)", obj)
        if p1:
            obj = obj.replace(p1.group(2),"")
            movie_rating_dict[obj]=obj_list[1]
        else:
            p2 = re.search(r"\((\d{4}|\?{4})[^()]*\)", obj)
            if not p2:
                obj = obj + " (????)"
            movie_rating_dict[obj]=obj_list[1]

pickle.dump(movie_rating_dict, open("movie_rating_dict.pkl", "wb" ))
            
with open("movie_rating_clean.txt", "w", encoding="utf-8") as outfile:
    for k, v in movie_rating_dict.items():
        outfile.write("%s,%s\n" %(k, v)) 

In [123]:
fname = "director_movies.txt"

director_movie_dict = {}
movie_director_dict = {}
line_count = 0

num = 0
with open(fname, encoding="ISO-8859-1") as fin:
    for line in fin:
        line_count += 1
        obj_list = re.split(r"\t+", line) ############# change to \t        
        if len(obj_list) < 2: ############### change to 11 later
            continue
        strip_list = []
        for obj in obj_list:
            strip_list.append(obj.strip("\n").strip())
        strip_list = list(filter(None, strip_list))
        obj_count = 0
        director_name = strip_list[0]
        director_name = director_name.replace(" ", "")
        if director_name not in director_movie_dict:
            new_movie_list = []
            director_movie_dict[director_name] = new_movie_list
 #       new_movie_list = []
 #       director_movie_dict[director_name] = new_movie_list

        for obj in strip_list:
            obj_count += 1

            if obj_count >= 2:
                p1 = re.search(r"\((\d{4}|\?{4})[^()]*\)(.+)", obj)
                if p1:
                        obj = obj.replace(p1.group(2),"")
                        ## movie 2 director dict update
                        if movie_director_dict.get(obj) == None:
                            new_director_list = []
                            new_director_list.append(director_name)
                            movie_director_dict[obj] = new_director_list
                        else:
                            movie_director_dict[obj].append(director_name)                        
                        ## director 2 movie dict update
                        director_movie_dict[director_name].append(obj)
                else:
                        p2 = re.search(r"\((\d{4}|\?{4})[^()]*\)", obj)
                        if not p2:
                            obj = obj + " (????)"
                        ## movie 2 director dict update
                        if movie_director_dict.get(obj) == None:
                            new_director_list = []
                            new_director_list.append(director_name)
                            movie_director_dict[obj] = new_director_list
                        else:
                            movie_director_dict[obj].append(director_name) 
                        ## director 2 movie dict update                        
                        director_movie_dict[director_name].append(obj)
                
pickle.dump(director_movie_dict, open("director_movie_dict.pkl", "wb" ))
pickle.dump(movie_director_dict, open("movie_director_dict.pkl", "wb" ))

with open("director_movie_clean.txt", "w", encoding="utf-8") as outfile:
    for k, v in director_movie_dict.items():
        for d in v:
            outfile.write("%s,%s\n" %(k, d)) 

In [124]:
director_rating_dict = {}
for key, value in director_movie_dict.items():
    sum_ratings = 0
    count = 0
    for mv in value:
        if mv in movie_rating_dict:
            sum_ratings += float(movie_rating_dict[mv])
            count += 1
    if count > 0:
        director_rating_dict[key] = sum_ratings/count

pickle.dump(director_rating_dict, open("director_rating_dict.pkl", "wb" ))

In [113]:
len(director_rating_dict)

113611

### Actor pagerank

In [48]:
from igraph import *

In [68]:
def mergetxt(fnames):
    with open("merged.txt", "w", encoding="ISO-8859-1") as outfile:
        for fname in fnames:
            with open(fname, encoding="ISO-8859-1") as infile:
                for line in infile:
                    outfile.write(line)
file_names = ["actor_movies.txt", "actress_movies.txt"]
mergetxt(file_names)

In [114]:
cast_movie_dict = {}
movie_cast_dict = {}
fname = "merged.txt"
line_count = 0
cast_count = 0
with open(fname, encoding="ISO-8859-1") as fin:
    for line in fin:
        line_count += 1
        obj_list = re.split(r"\t+", line) ############# change to \t
        if len(obj_list) < 11: ############### change to 11 later
            continue
        cast_count += 1
        strip_list = []
        for obj in obj_list:
            strip_list.append(obj.strip("\n").strip())
        strip_list = list(filter(None, strip_list))
        obj_count = 0
        cast_name = strip_list[0]
        cast_name = cast_name.replace(" ", "")
        new_movie_list = []
        cast_movie_dict[cast_name] = new_movie_list
        for obj in strip_list:
            obj_count += 1
            if obj_count >= 2:
                p1 = re.search(r"\((\d{4}|\?{4})[^()]*\)(.+)", obj)
                if p1:
                    obj = obj.replace(p1.group(2),"")
                    ## movie 2 cast dict update
                    if obj in movie_cast_dict:
                        movie_cast_dict[obj].append(cast_name)
                    else:
                        new_cast_list = []
                        new_cast_list.append(cast_name)
                        movie_cast_dict[obj] = new_cast_list
                    ## cast 2 movie dict update
                    cast_movie_dict[cast_name].append(obj)
                else:
                    p2 = re.search(r"\((\d{4}|\?{4})[^()]*\)", obj)
                    if not p2:
                        obj = obj + " (????)"
                    # movie 2 cast dict update
                    if obj in movie_cast_dict:
                        movie_cast_dict[obj].append(cast_name)
                    else:
                        new_cast_list = []
                        new_cast_list.append(cast_name)
                        movie_cast_dict[obj] = new_cast_list
                    ## cast 2 movie dict update
                    cast_movie_dict[cast_name].append(obj)
print("There are ",len(cast_movie_dict)," actors/actresses and ",len(movie_cast_dict)," unique movies...")

pickle.dump(cast_movie_dict, open("cast_movie_dict.pkl", "wb" ))
pickle.dump(movie_cast_dict, open("movie_cast_dict.pkl", "wb" ))

## create edge list for actor/actress network
dic = {}
edgelist_dict = {}
for c1, ml in cast_movie_dict.items():
    for m in ml:
        for c2 in movie_cast_dict[m]:
            if c1 != c2:
                dic[c1] = 1
                dic[c2] = 1
                key = c1 + "\n" + c2
                if edgelist_dict.get(key) == None:
                    edgelist_dict[key] = 1.0/len(ml)
                else:
                    edgelist_dict[key] += 1.0/len(ml)
print("There are ",len(dic)," nodes in the cast network...")

with open("cast_network_edgelist.txt", "w", encoding="utf-8") as outfile:
    for k, v in edgelist_dict.items():
        c1, c2 = k.split("\n")
        line = c1 + '\t' + c2 + '\t' + str(v) + '\n'
#        line = ",".join([c1.replace(",", ""), c2.replace(",", ""), str(v)]) + "\n"
        outfile.write(line)

There are  113129  actors/actresses and  468244  unique movies...
There are  113050  nodes in the cast network...


In [115]:
g = Graph.Read_Ncol('cast_network_edgelist.txt', directed=True)
page_rank = g.pagerank(vertices=None, directed=True)
actor_pagerank_dict = {}
for idx, v in enumerate(g.vs):
    actor_pagerank_dict[v["name"]] = page_rank[idx]
    
pickle.dump(actor_pagerank_dict, open("actor_pagerank_dict.pkl", "wb" ))

### Build features

In [2]:
actor_pagerank_dict = pickle.load(open("actor_pagerank_dict.pkl", "rb" ))
director_rating_dict = pickle.load(open("director_rating_dict.pkl", "rb" ))
cast_movie_dict = pickle.load(open("cast_movie_dict.pkl", "rb" ))
movie_cast_dict = pickle.load(open("movie_cast_dict.pkl", "rb" ))
director_movie_dict = pickle.load(open("director_movie_dict.pkl", "rb" ))
movie_director_dict = pickle.load(open("movie_director_dict.pkl", "rb" ))
movie_rating_dict = pickle.load(open("movie_rating_dict.pkl", "rb" ))

In [3]:
movie_10cast_dict = {}

for mv, cast in movie_cast_dict.items():
    if len(cast)>9:
        movie_10cast_dict[mv] = cast

movie_with_CR = set(movie_10cast_dict.keys()).intersection(movie_rating_dict.keys())
avail_movies = set(movie_with_CR).intersection(movie_director_dict.keys())

In [4]:
len(avail_movies)

83537

In [10]:
def build_feature(movie):
    features = [0] * 6
    actors = movie_10cast_dict[movie]
    page_ranks = []
    for actor in actors:
        page_ranks.append(actor_pagerank_dict[actor])
    sorted_page_ranks = sorted(page_ranks, reverse=True)
    features[:5] = sorted_page_ranks[:5]
    
    directors = movie_director_dict[movie]
    director_ratings = 0
    count = 0
    for director in directors:
        if director in director_rating_dict:
            director_ratings += float(director_rating_dict[director])
            count += 1
    features[5] = director_ratings/count
    return np.array(features)

feature_vector = np.vstack(map(build_feature, timer(avail_movies)))

100%|██████████| 83537/83537 [00:02<00:00, 28739.38it/s]


In [11]:
actual_rating = []
for movie in timer(avail_movies):
    actual_rating.append(float(movie_rating_dict[movie]))
rating_vector = np.array(actual_rating)

100%|██████████| 83537/83537 [00:00<00:00, 377217.06it/s]


### Linear Regression model training

In [12]:
model = linear_model.LinearRegression()
model.fit(feature_vector, rating_vector)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
test_movies = [
    "Batman v Superman: Dawn of Justice (2016)",
    "Mission: Impossible - Rogue Nation (2015)",
    "Minions (2015)"
]

In [14]:
for mv in test_movies:
    print ("%s: %.2f" %(mv,model.predict([build_feature(mv)])))

Batman v Superman: Dawn of Justice (2016): 7.26
Mission: Impossible - Rogue Nation (2015): 6.84
Minions (2015): 7.17


### Error calculation

In [15]:
prediction = model.predict(feature_vector)

In [16]:
mcs.mean_squared_error(rating_vector, prediction)

0.6881090242779832